In [1]:
import pandas as pd
import sqldf
import plotly.express as px
import numpy as np

import warnings
warnings.filterwarnings('ignore')

### Data download

In [2]:
df_r = pd.read_csv('data_markets.csv')

In [3]:
df = df_r.copy()

In [4]:
df.describe()

,Unnamed: 0,SumOrderOnTheWebsite,SumDocument,Quantity,Price,SumLine,PurchasePrice,MonthOfTheOrderDate,YearOfOrderDate,Margin,SumOfServices,SumOfDelivery,NumberLine,QuantitySoldToCustomer
count,1.247887e+06,1.247887e+06,1.247887e+06,1.247887e+06,1.247887e+06,1.247887e+06,1.242661e+06,1.247887e+06,1247887.0,1.242661e+06,1.247887e+06,1.247887e+06,1.247887e+06,1.247887e+06
mean,3.120115e+05,4.615599e+03,4.134379e+03,1.491944e+00,8.861485e+02,9.583688e+02,6.678611e+02,4.496495e+00,2017.0,2.162378e+02,7.767476e+01,7.323472e+01,5.684645e+00,1.221355e+00
std,1.801868e+05,6.035081e+03,5.240028e+03,2.117530e+00,2.326047e+03,2.412666e+03,1.725737e+03,1.100216e+00,0.0,7.320686e+02,2.714660e+02,2.680414e+02,7.316841e+00,2.023867e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.500000e-01,3.000000e+00,2017.0,-8.106000e+04,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
25%,1.559855e+05,1.797000e+03,1.422000e+03,1.000000e+00,9.300000e+01,1.150000e+02,7.254000e+01,4.000000e+00,2017.0,1.072000e+01,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
50%,3.119710e+05,2.991000e+03,2.713000e+03,1.000000e+00,2.880000e+02,2.990000e+02,1.999000e+02,5.000000e+00,2017.0,4.964000e+01,4.900000e+01,4.900000e+01,3.000000e+00,1.000000e+00
75%,4.679570e+05,5.324000e+03,4.978000e+03,1.000000e+00,8.500000e+02,8.990000e+02,6.477000e+02,5.000000e+00,2017.0,1.654300e+02,1.500000e+02,1.500000e+02,7.000000e+00,1.000000e+00
max,6.310330e+05,7.447660e+05,2.035760e+05,1.840000e+02,9.244200e+04,3.459000e+05,8.038437e+04,6.000000e+00,2017.0,7.946100e+04,1.159000e+05,1.159000e+05,1.470000e+02,1.840000e+02


### Data preprocessing

In [5]:
#deleting rows with zeros in phone number column
df = df[df['Telephone_new'] != '0']

##### Creating df with orders

In [6]:
orders_q = """ 
select 
    OrderNumberOnTheWebsite,
    sum(Quantity) as NumberOfItemsInOrder,
    max(SumOrderOnTheWebsite) as SumOrder,
    max(Telephone_new) as client_id,
    sum(Price) as RevenueOrder,
    sum(Margin) as MarginOrder,
    case when NewStatus = 'Доставлен' and Cancelled = 'Нет' then true else false end as is_purchased,
    sum(case when NewStatus = 'Доставлен' and Cancelled = 'Нет' then Quantity else null end) as cnt_purchased_items
from df
group by 1
"""

In [7]:
orders = sqldf.run(orders_q)
orders.head(5)

,OrderNumberOnTheWebsite,NumberOfItemsInOrder,SumOrder,client_id,RevenueOrder,MarginOrder,is_purchased,cnt_purchased_items
0,1303000307_TR,1,3900,55575057-54525157495471,3900.0,1600.00,0,NaN
1,1303000308_TR,1,3199,55575057-54525157495471,3199.0,676.70,0,NaN
2,1303000309_TR,1,10900,55575656-51545655485571,10900.0,4067.00,0,NaN
3,1303000310_TR,1,54900,55575656-51545655485571,54900.0,10970.34,0,NaN
4,1303000311_TR,4,6272,55575656-51545655485571,6272.0,1809.18,0,NaN


##### Summarizing information about clients

In [8]:
query = """
select 
    client_id,
    count(distinct OrderNumberOnTheWebsite) as DistinctCountOrders,
    avg(SumOrder) as AverageOrderSum,
    avg(NumberOfItemsInOrder) as AverageNumItemsInOrder,
    sum(RevenueOrder) as RevenueClient,
    sum(NumberOfItemsInOrder) as TotalNumberItemsOrdered,
    avg(MarginOrder) as AverageMarginOrder,
    count(case when is_purchased is true then OrderNumberOnTheWebsite else null end) as cnt_purchased_orders,
    avg(case when is_purchased is true then SumOrder else null end) as AvgOrderSumPurchased,
    avg(case when is_purchased is true then NumberOfItemsInOrder else null end) as AvgNumItemsOrderPurchased,
    sum(case when is_purchased is true then RevenueOrder else null end) as RevenueClientPurchased,
    sum(case when is_purchased is true then cnt_purchased_items else null end) as PurchasedItems,
    avg(case when is_purchased is true then MarginOrder else null end) as AvgMarginPurchased
    
from 
    orders
group by 
    1
order by 
    cnt_purchased_orders desc
"""

In [9]:
res = sqldf.run(query)

In [10]:
res['SharePurchasedOrders'] = res['cnt_purchased_orders'] / res['DistinctCountOrders']
res['AverageCheck'] = res['RevenueClient'] / res['DistinctCountOrders']
res['SharePurchasedOrders'] = res['SharePurchasedOrders'].apply(lambda x: round(x,2))
res.head(5)

,client_id,DistinctCountOrders,AverageOrderSum,AverageNumItemsInOrder,RevenueClient,TotalNumberItemsOrdered,AverageMarginOrder,cnt_purchased_orders,AvgOrderSumPurchased,AvgNumItemsOrderPurchased,RevenueClientPurchased,PurchasedItems,AvgMarginPurchased,SharePurchasedOrders,AverageCheck
0,55575053-54505550565678,100,1758.900000,2.440000,172280.0,244,219.014700,80,1574.750000,2.275000,124703.0,182.0,183.293250,0.80,1722.800000
1,55575048-48524956565375,66,580.318182,3.787879,34598.0,250,144.108615,60,542.216667,2.866667,31004.0,171.0,132.661000,0.91,524.212121
2,55575555-57524953505673,54,823.166667,2.703704,43574.0,146,95.811296,53,816.169811,2.679245,42380.0,142.0,93.149057,0.98,806.925926
3,55574953-50505150484978,59,3608.406780,4.169492,205490.0,246,429.037119,45,3217.244444,2.222222,137601.0,100.0,391.009556,0.76,3482.881356
4,55575054-51515151484875,46,2669.847826,1.717391,121083.0,79,310.657174,40,2689.475000,1.800000,105849.0,72.0,314.361500,0.87,2632.239130


In [11]:
q = """select cnt_purchased_orders, count(*) from res group by 1 order by 1 desc"""
rr = sqldf.run(q)

In [12]:
df = df.rename(columns={"Telephone_new": "client_id"})
df['client_id'] = df['client_id'].astype('str') 

#### Creating pivot table with the quantity of imems bought by a client in each group2

In [13]:
group2 = df[['client_id', 'Group2', 'Quantity']]

In [14]:
group2_pivot = pd.pivot_table(group2, values='Quantity', index=['client_id'],
                    columns=['Group2'], aggfunc=np.sum, fill_value=0)

In [15]:
group2_pivot["total"] = group2_pivot.sum(axis=1)
group2_pivot = group2_pivot[group2_pivot['total'] > 0]
group2_pivot = group2_pivot.iloc[:,:].div(group2_pivot.total, axis=0)
group2_pivot = group2_pivot.drop(['total'], axis=1)
group2_pivot = group2_pivot.reset_index()
group2_pivot.head(5)

Group2,client_id,ДЕТСКОЕ ПИТАНИЕ,ЖЕНСКИЕ ШТУЧКИ,ИГРУШКИ,"КАНЦТОВАРЫ, КНИГИ, ДИСКИ",КОСМЕТИКА/ГИГИЕНА,КРУПНОГАБАРИТНЫЙ ТОВАР,ОБУВЬ,ПОДГУЗНИКИ,СОПУТСТВУЮЩИЕ ТОВАРЫ,"ТЕКСТИЛЬ, ТРИКОТАЖ",ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА,ТОВАРЫ ДЛЯ ЖИВОТНЫХ,ТОВАРЫ ДЛЯ КОРМЛЕНИЯ
0,32555749-545749525150 .,0.0,0.0,0.00,1.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000
1,49484949-49494949494911,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,0.0,0.0,0.1875
2,52535348-48484848484840,0.0,0.0,0.00,0.0,0.0,0.0,0.0,1.0000,0.0,0.0,0.0,0.0,0.0000
3,52575355-48514856495345,0.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000
4,54545548-48484848484860,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,1.0,0.0,0.0,0.0000


#### Creating pivot table with the quantity of imems bought by a client in each type

In [16]:
types = df[['client_id', 'Type', 'Quantity']]

In [17]:
types_pivot = pd.pivot_table(types, values='Quantity', index=['client_id'],
                    columns=['Type'], aggfunc=np.sum, fill_value=0)

In [18]:
types_pivot["total"] = types_pivot.sum(axis=1)
types_pivot = types_pivot[types_pivot['total'] > 0]
types_pivot = types_pivot.iloc[:,:].div(types_pivot.total, axis=0)
types_pivot = types_pivot.drop(['total'], axis=1)
types_pivot = types_pivot.reset_index()
types_pivot.head(5)

Type,client_id,ИГРУШКИ,ИНОЕ,КГТ,ОДЕЖДА,ППКП
0,32555749-545749525150 .,1.00,0.0,0.0,0.0,0.00
1,49484949-49494949494911,0.75,0.0,0.0,0.0,0.25
2,52535348-48484848484840,0.00,0.0,0.0,0.0,1.00
3,52575355-48514856495345,1.00,0.0,0.0,0.0,0.00
4,54545548-48484848484860,0.00,0.0,0.0,1.0,0.00


##### Creating pivot table with the quantity of imems bought by a client in each group3

In [19]:
group3 = df[['client_id', 'Group3', 'Quantity']]

In [20]:
group3_pivot = pd.pivot_table(group3, values='Quantity', index=['client_id'],
                    columns=['Group3'], aggfunc=np.sum, fill_value=0)

In [21]:
group3_pivot["total"] = group3_pivot.sum(axis=1)
group3_pivot = group3_pivot[group3_pivot['total'] > 0]
group3_pivot = group3_pivot.iloc[:,:].div(group3_pivot.total, axis=0)
group3_pivot = group3_pivot.drop(['total'], axis=1)
group3_pivot = group3_pivot.reset_index()
group3_pivot.head(5)

Group3,client_id,АВТОКРЕСЛА,АКСЕССУАРЫ,АКСЕССУАРЫ ДЛЯ ВОЛОС,АКСЕССУАРЫ ДЛЯ ДЕВОЧЕК,"АКСЕССУАРЫ МАКИЯЖ, МАНИКЮР",АКТИВНЫЙ ОТДЫХ,БАКАЛЕЯ,БИЖУТЕРИЯ,БУТЫЛОЧКИ,...,ТРУСИКИ НЕПРОМОКАЕМЫЕ,"ФИГУРЫ, ПЕРСОНАЖИ",ФОТОАЛЬБОМЫ/ФОТОРАМКИ,ХОДУНКИ/ПРЫГУНКИ,ХОЗЯЙСТВЕННЫЕ ТОВАРЫ,ЧАСЫ/ОЧКИ,"ШАПКИ, ШАРФЫ",ЭЛЕКТРОМОБИЛИ/МАШИНЫ ПЕДАЛЬНЫЕ,ЭЛЕКТРОПРИБОРЫ,ЭЛЕМЕНТЫ ПИТАНИЯ
0,32555749-545749525150 .,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49484949-49494949494911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1875,...,0.0,0.625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,52535348-48484848484840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,52575355-48514856495345,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,54545548-48484848484860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Creating pivot table with the quantity of imems bought by a client in each Region

In [22]:
regions = df[['client_id', 'Region_max', 'Quantity']]

In [23]:
regions_pivot = pd.pivot_table(regions, values='Quantity', index=['client_id'],
                    columns=['Region_max'], aggfunc=np.sum, fill_value=0)

In [24]:
regions_pivot["total"] = regions_pivot.sum(axis=1)
regions_pivot = regions_pivot[regions_pivot['total'] > 0]
regions_pivot = regions_pivot.iloc[:,:].div(regions_pivot.total, axis=0)
regions_pivot = regions_pivot.drop(['total'], axis=1)
regions_pivot = regions_pivot.reset_index()
regions_pivot.head(5)

Region_max,client_id,CENTRAL,FAR EAST,NORTH,PRIVOLZIE,SIBERIA,SOUTHERN,URAL
0,32555749-545749525150 .,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49484949-49494949494911,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,52535348-48484848484840,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,52575355-48514856495345,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,54545548-48484848484860,1.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Combining all the tables together and getting our data mart

In [25]:
qj = """ 
select *
from res 
join group2_pivot on group2_pivot.client_id = res.client_id
join types_pivot on types_pivot.client_id = res.client_id
join group3_pivot on group3_pivot.client_id = res.client_id
join regions_pivot on regions_pivot.client_id = res.client_id
"""

In [26]:
ans = sqldf.run(qj)
ans

,index,client_id,DistinctCountOrders,AverageOrderSum,AverageNumItemsInOrder,RevenueClient,TotalNumberItemsOrdered,AverageMarginOrder,cnt_purchased_orders,AvgOrderSumPurchased,...,ЭЛЕМЕНТЫ ПИТАНИЯ,index,client_id,CENTRAL,FAR EAST,NORTH,PRIVOLZIE,SIBERIA,SOUTHERN,URAL
0,0,55575053-54505550565678,100,1758.900000,2.440000,172280.0,244,219.014700,80,1574.750000,...,0.0,112277,55575053-54505550565678,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,55575048-48524956565375,66,580.318182,3.787879,34598.0,250,144.108615,60,542.216667,...,0.0,91753,55575048-48524956565375,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2,55575555-57524953505673,54,823.166667,2.703704,43574.0,146,95.811296,53,816.169811,...,0.0,179926,55575555-57524953505673,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,55574953-50505150484978,59,3608.406780,4.169492,205490.0,246,429.037119,45,3217.244444,...,0.0,58950,55574953-50505150484978,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,55575054-51515151484875,46,2669.847826,1.717391,121083.0,79,310.657174,40,2689.475000,...,0.0,121461,55575054-51515151484875,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194000,199173,55484848-48484848484870,3,748.000000,1.000000,1896.0,3,172.316667,0,NaN,...,0.0,5,55484848-48484848484870,1.0,0.0,0.0,0.0,0.0,0.0,0.0
194001,199174,54545548-48484848484860,1,3067.000000,3.000000,2567.0,3,891.450000,0,NaN,...,0.0,4,54545548-48484848484860,1.0,0.0,0.0,0.0,0.0,0.0,0.0
194002,199175,52575355-48514856495345,1,2794.000000,6.000000,2015.0,6,649.450000,0,NaN,...,0.0,3,52575355-48514856495345,0.0,0.0,0.0,0.0,1.0,0.0,0.0
194003,199176,52535348-48484848484840,1,1139.000000,1.000000,1090.0,1,110.000000,0,NaN,...,0.0,2,52535348-48484848484840,1.0,0.0,0.0,0.0,0.0,0.0,0.0
